In [16]:
pip install unidecode

In [17]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import re
import time
import pandas as pd
import os
from unidecode import unidecode
myheader = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",  "Accept":"text/html,application/xhtml+xml,application/xml; q=0.9,image/webp,image/apng,*/*;q=0.8"}

In [18]:
def extractLinks(root_url, pattern=None):

  try:
    html = requests.get(root_url, headers=myheader)
    html.raise_for_status()

    # Create a BS object to parse the HTML content
    bsObj = BeautifulSoup(html.content, "lxml")

    links = set()

    # Iterate through all 'a' tags with 'href' attribute
    for link in bsObj.find_all("a", href=True):
      href = link["href"]

      # Check if the link matches the specified pattern using regular expressions('http://' or 'https://'))
      if re.match(pattern, href) or pattern is None:
        if href.startswith(("http://", "https://", "www.")):
          links.add(href)
        else:
          links.add(urljoin(root_url, href))

    return links

  #Exception handling
  except requests.RequestException as e:
    print(f"Error during HTTP request for {root_url}: {e}")
    return set()

  except Exception as e:
    print(f"An unexpected error occurred for {root_url}: {e}")
    return set()


In [19]:
def scrapeAndSave(links, category, element, classOrId, find_all_flag=False):
    articleSet = set()

    for link in links:
        try:
            article = requests.get(link, headers=myheader)
            article.raise_for_status()

            soup = BeautifulSoup(article.content, "html.parser")

            if classOrId == "class":
                if find_all_flag:
                    bodyElements = soup.find_all(class_=element)
                else:
                    bodyElements = [soup.find(class_=element)]
            else:
                if find_all_flag:
                    bodyElements = soup.find_all(id=element)
                else:
                    bodyElements = [soup.find(id=element)]

            for bodyElement in bodyElements:
                if bodyElement:
                    # If the element exists, extract its text
                    bodyText = bodyElement.text
                    cleanedText = unidecode(bodyText).strip()
                    articleSet.add((cleanedText, category, link))

                else:
                    print(f"Could not find body element for link: {link}")
                    articleSet.add(("Not available", "N/A", link))

            time.sleep(1)

        except requests.RequestException as e:
            print(f"Error during HTTP request for link {link}: {e}")
            continue

        except Exception as e:
            print(f"An unexpected error occurred for link {link}: {e}")
            continue

    return articleSet

In [20]:
def convertToCsv(data, filename):
    file_exists = os.path.exists(filename)

    # If the file exists, read it into a DataFrame
    if file_exists:
        existing_df = pd.read_csv(filename)
    else:
        existing_df = pd.DataFrame(columns=["Description"])

    # New DataFrame with the new data
    new_df = pd.DataFrame({"Description": [info[0] for info in data],
                           "Department": [info[1] for info in data],
                           "Link": [info[2] for info in data]})

    is_duplicate = new_df.isin(existing_df.to_dict(orient='list')).all(axis=1).any()

    # Update the new data to the existing DataFrame and write to CSV
    if not is_duplicate:
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
        combined_df.to_csv(filename, index=False, mode='w' if file_exists else 'a', header=not file_exists)
    else:
        print("Data is already present in the CSV file. Not adding duplicates.")

In [21]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_website(url):
    try:
        # Fetch
        response = requests.get(url)
        response.raise_for_status()

        # Parse
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract text from all the paragraphs excluding those containing "Read more" or "Continue reading"
        paragraphs = soup.find_all('p')
        extracted_text = '\n'.join([p.get_text() for p in paragraphs if "Read More..." not in p.get_text() and "Continue reading" not in p.get_text()])

        return extracted_text

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

url = "https://www.bbc.com/news/topics/c008ql15vpyt"
extracted_text = extract_text_from_website(url)
if extracted_text:
    print(extracted_text)
else:
    print("Failed to extract text from the website.")


The Taliban accuses Pakistan of killing eight women and children after an attack killed Pakistani troops.
Provides an overview of Pakistan, including key dates and facts about this south Asian country.
Judges in Pakistan say he shared blasphemous pictures and videos to outrage the feelings of Muslims.
Seven years after Ishrat Baig donated £40k to a charity to build an orphanage, it is still incomplete.
The icy conditions are unexpected as the region is typically humid in March.
Mr Sharif got parliament's backing after a deal shut out supporters of his jailed predecessor Imran Khan.
Pakistan's army has long wielded an unchallenged amount of power over politics - until now.
Hundreds mistakenly accused the woman of wearing a dress adorned with Quran verses.
It comes after a contentious election where the largest party led by former-PM Imran Khan alleged vote-rigging.
Copyright 2024 BBC. All rights reserved.  The BBC is not responsible for the content of external sites. Read about our appr

In [22]:
import requests
from bs4 import BeautifulSoup
import csv

def extract_text_from_website(url):
    try:
        # Fetch
        response = requests.get(url)
        response.raise_for_status()

        # Parse
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract text from all the paragraphs excluding those containing "Read more" or "Continue reading"
        paragraphs = soup.find_all('p')
        extracted_text = '\n'.join([p.get_text() for p in paragraphs if "Read More..." not in p.get_text() and "Continue reading" not in p.get_text()])

        return extracted_text

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def save_to_csv(data, filename='extracted_text.csv'):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Extracted Text'])
            for row in data.split('\n'):
                writer.writerow([row])
        print(f"Data saved to {filename} successfully.")
    except Exception as e:
        print(f"An error occurred while saving to CSV: {e}")

url = "https://www.bbc.com/news/topics/c008ql15vpyt"
extracted_text = extract_text_from_website(url)
if extracted_text:
    save_to_csv(extracted_text)
else:
    print("Failed to extract text from the website.")

Data saved to extracted_text.csv successfully.
